In [35]:
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# File to Load (Remember to Change These)
train_data = "cleaned_train.csv"

In [36]:
df_train_clean = pd.read_csv(train_data)

In [37]:
df_train_clean[0:5]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,country_of_residence_United States,country_of_residence_Viet Nam,used_app_before_no,used_app_before_yes,test_taker_?,test_taker_Health care professional,test_taker_Others,test_taker_Parent,test_taker_Relative,test_taker_Self
0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
# Split our preprocessed data into our features and target arrays
y = df_train_clean.Autism_Diagnosis.values
X = df_train_clean.drop(columns="Autism_Diagnosis").values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Optimization attempt 1

- more neurons in hidden layers
- additional hidden layer
- second hidden layer changed to 'sigmoid'

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]


nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim= input_features))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# Third hidden layer
nn_1.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_1.fit(X_train_scaled, y_train, epochs=150)

model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               6600      
                                                                 
 dense_11 (Dense)            (None, 50)                5050      
                                                                 
 dense_12 (Dense)            (None, 20)                1020      
                                                                 
 dense_13 (Dense)            (None, 1)                 21        
                                                                 
Total params: 12,691
Trainable params: 12,691
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
19/19 [==============================] - 1s 4ms/step - loss: 0.5178 - accuracy: 0.7733
Epoch 2/150
19/19 [==============================] - 0s 4ms/step - loss: 0.4128 - a

In [44]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 8.0 MB/s eta 0:00:00


In [45]:
# Create a method that creates a new Sequential model with hyperparameter options

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value= 100,
        step=10), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=50,
            step=10),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

  # Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

# Top 3 model hyperparameters 
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
  print(param.values)

# Top 3 models
top_model = tuner.get_best_models(3)
for model in top_model:
  model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
  print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Trial 180 Complete [00h 00m 07s]
val_accuracy: 0.8849999904632568

Best val_accuracy So Far: 0.8899999856948853
Total elapsed time: 00h 10m 21s
{'activation': 'relu', 'first_units': 21, 'num_layers': 5, 'units_0': 41, 'units_1': 31, 'units_2': 11, 'units_3': 31, 'units_4': 31, 'tuner/epochs': 6, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 81, 'num_layers': 1, 'units_0': 41, 'units_1': 31, 'units_2': 41, 'units_3': 21, 'units_4': 41, 'tuner/epochs': 17, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 81, 'num_layers': 1, 'units_0': 41, 'units_1': 31, 'units_2': 41, 'units_3': 21, 'units_4': 41, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0078'}
7/7 - 0s - loss: 0.3005 - accuracy: 0.8900 - 277ms/epoch - 40ms/step
Loss: 0.30053573846817017, Accuracy: 0.8899999856948853
7/7 - 0s - loss: 0.3398 - accuracy: 0.8900 - 230ms/

In [46]:
# Export our model to HDF5 file
best_model.save("Neural_Network_Optimized_Standard.h5")